### Run these before running anything else and pick path file for data

In [24]:
using Pkg
using Plots
using Random
using DataFrames
using CSV

In [23]:
userpath = "/Users/justin_tackett/Documents"

"/Users/justin_tackett/Documents"

In [18]:
#First time run, not necessary to run after running it once on machine

Pkg.add("DataFrames")
Pkg.add("CSV")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


# Distributed N-Body

In [29]:
## User Data

usernbody  = 100 
usermmin   = 1
usermmax   = 2
usermaxvel = 100
userxmin   = -100
userxmax   = 100
userymin   = -100
userymax   = 100
userzmin   = -100
userzmax   = 100
usergravbound = true
userG = 1
userc = 1
KtoP  = .6 #The ratio of the totel kinetic energy to total potential energy; must be less than one, if too close to one objects may fly off


"/Users/justin_tackett/Documents"

In [27]:
function R(x1,y1,z1,x2,y2,z2,ndim)
    if(ndim==2)
        return ((x2-x1)^2+(y2-y1)^2+(z2-z1)^2)^(1/2)
    elseif(ndim==3)
        return ((x2-x1)^2+(y2-y1)^2)^(1/2)
    else
        println("Error with function R, ndim not valid")
    end
    
end

function V(vx,vy,vz,ndim)
    if(ndim==2)
        return ((vx)^2+(vy)^2+(vz)^2)^(1/2)
    elseif(ndim==3)
        return ((vx)^2+(vy)^2)^(1/2)
    else
        println("Error with function R, ndim not valid")
    end
    
end

function KE(m,vx,vy,vz,ndim)
    return .5*m*(V(vx,vy,vz,ndim)^2)
end

function PE(x1,y1,z1,x2,y2,z2,ndim,G,m1,m2)
    return G*m1*m2/R(x1,y1,z1,x2,y2,z2,ndim)
end



    



function generateRandomNBodies(nbody, mmin, mmax, maxvel, xmin, xmax, ymin, ymax, zmin=0, zmax=0, gravbound::Bool=true, seed=1234)
    xvals  = zeros(nbody)
    yvals  = zeros(nbody)
    vxvals = zeros(nbody)
    vyvals = zeros(nbody)
    mvals  = zeros(nbody)
    pxvals = zeros(nbody)
    pyvals = zeros(nbody)
    pzvals = zeros(nbody)
    totpot = 0
    totkin = 0
    
    #Determine number of dimensions
    ##    
    if(zmin==0 && zmax==0)
        ndim = 2
    else
        ndim = 3
    end
    
    if(ndim==3)
        zvals  = zeros(nbody)
        vzvals = zeros(nbody)
    end
    
    #Determine if virial theorem should be used
        
    if(gravbound==true)
        
        #Virial Theorem FIX_ME
        #Rescale Velocities
        #Check for velocity distributions in clusters
        #Trial and error to find what Kin En is too small
        #Look into ang momentum
        #Look into gravitational stability of many bodies
        #Numerical Recipes, look for random generation on distribution
        #Transform to center of mass frame
        #Set up with circluar orbits
        
        #Two euler angles, semi major axis, eccentricity
        #Conventions by intrinsic rotations
            
        rng = MersenneTwister(seed)
        if(ndim==2)
            for i in 1:nbody
                xvals[i]  = rand(rng, Float64)*(xmax-xmin)+xmin
                yvals[i]  = rand(rng, Float64)*(ymax-ymin)+ymin
                mvals[i]  = rand(rng, Float64)*(mmax-mmin)+mmin
                vxvals[i] = rand(rng, Float64)*(maxvel/(2^(1/2)))*2-maxvel/(2^(1/2))
                vyvals[i] = rand(rng, Float64)*(maxvel/(2^(1/2)))*2-maxvel/(2^(1/2))
            end
        elseif(ndim==3)
            for i in 1:nbody
                xvals[i]  = rand(rng, Float64)*(xmax-xmin)+xmin
                yvals[i]  = rand(rng, Float64)*(ymax-ymin)+ymin
                zvals[i]  = rand(rng, Float64)*(zmax-zmin)+zmin
                mvals[i]  = rand(rng, Float64)*(mmax-mmin)+mmin
                vxvals[i] = rand(rng, Float64)*(maxvel/(3^(1/2)))*2-maxvel/(3^(1/2)) #If something goes wrong, check these first
                vyvals[i] = rand(rng, Float64)*(maxvel/(3^(1/2)))*2-maxvel/(3^(1/2))
                vzvals[i] = rand(rng, Float64)*(maxvel/(3^(1/2)))*2-maxvel/(3^(1/2))
            end
        else
            println("Error with Random Generation, ndim not passed correctly, actual value ",ndim)
        end
        
        for i in 1:nbody
            if(ndim==2)
                totkin = totkin + KE(mvals[i],vxvals[i],vyvals[i],0,ndim)
            elseif(ndim==3)
                totkin = totkin + KE(mvals[i],vxvals[i],vyvals[i],vzvals[i],ndim)
            end
        end
        
        for i in 1:(nbody - 1)
            for j in (i+1):nbody
                if(ndim==2)
                    totpot = totpot + PE(xvals[i],yvals[i],0,xvals[j],yvals[j],0,ndim,userG,mvals[i],mvals[j])
                elseif(ndim==3)
                    totpot = totpot + PE(xvals[i],yvals[i],zvals[i],xvals[j],yvals[j],zvals[j],ndim,userG,mvals[i],mvals[j])
                end
            end
        end
        
        vmod = sqrt(KtoP*totpot/totkin)
        
        vxvals = vxvals * vmod
        vyvals = vyvals * vmod
        vzvals = vzvals * vmod
        
        
                    
    else
        
        #Random generation
        #To add later:
        # -Normal randomization option for RNG
            
        rng = MersenneTwister(seed)
        if(ndim==2)
            for i in 1:nbody
                xvals[i]  = rand(rng, Float64)*(xmax-xmin)+xmin
                yvals[i]  = rand(rng, Float64)*(ymax-ymin)+ymin
                mvals[i]  = rand(rng, Float64)*(mmax-mmin)+mmin
                vxvals[i] = rand(rng, Float64)*(maxvel/(2^(1/2)))*2-maxvel/(2^(1/2))
                vyvals[i] = rand(rng, Float64)*(maxvel/(2^(1/2)))*2-maxvel/(2^(1/2))
            end
        elseif(ndim==3)
            for i in 1:nbody
                xvals[i]  = rand(rng, Float64)*(xmax-xmin)+xmin
                yvals[i]  = rand(rng, Float64)*(ymax-ymin)+ymin
                zvals[i]  = rand(rng, Float64)*(zmax-zmin)+zmin
                mvals[i]  = rand(rng, Float64)*(mmax-mmin)+mmin
                vxvals[i] = rand(rng, Float64)*(maxvel/(3^(1/2)))*2-maxvel/(3^(1/2)) #If something goes wrong, check these first
                vyvals[i] = rand(rng, Float64)*(maxvel/(3^(1/2)))*2-maxvel/(3^(1/2))
                vzvals[i] = rand(rng, Float64)*(maxvel/(3^(1/2)))*2-maxvel/(3^(1/2))
            end
        else
            println("Error with Random Generation, ndim not passed correctly, actual value ",ndim)
        end
        
    end
    if(ndim==2)
        for i in 1:nbody
            pxvals[i] = vxvals[i] * mvals[i]
            pyvals[i] = vyvals[i] * mvals[i]
        end
    elseif(ndim==3)
        for i in 1:nbody
            pxvals[i] = vxvals[i] * mvals[i]
            pyvals[i] = vyvals[i] * mvals[i]
            pzvals[i] = vzvals[i] * mvals[i]
        end
    end
    
    
    #Mass, coodrinates, momenta    
    if(ndim==2)
        df = DataFrame(x = xvals,
                        y  = yvals,
                        px = pxvals,
                        py = pyvals,
                        )
    elseif(ndim==3)
        df = DataFrame(x = xvals,
                        y  = yvals,
                        z  = zvals,
                        px = pxvals,
                        py = pyvals,
                        pz = pzvals,
                        )
    end
    
    CSV.write(userpath * "/nbodyinitialData.csv", df)
end

generateRandomNBodies (generic function with 5 methods)

In [31]:
generateRandomNBodies(usernbody, usermmin, usermmax, usermaxvel, userxmin, userxmax, userymin, userymax, userzmin, userzmax, usergravbound, 1234)

"/Users/justin_tackett/Documents/nbodyinitialData.csv"

# 3 Body (Non-Kozai)

In [10]:
m1 = 1
m2 = 1
m3 = 1 #Elliptical object
userG = 1
CR = 10 #Circular orbit R, how far are m1 and m2 from each other initially, in code unites
ER = 10 #Distance of the third, elliptical object from the center of mass
theta = 45; #Incline of mass 3 in degrees up from the orbital plane
ecc = .5 #Eccentricity

"/Users/justin_tackett/Documents"

In [12]:
v1x = 0
v2x = 0
v1y = sqrt(userG*m2/CR);
v2y = -sqrt(userG*m1/CR);
v1z = 0
v2z = 0
x1 = CR/2
x2 = -CR/2
y1 = 0
y2 = 0
z1 = 0
z2 = 0

x3 = 0
y3 = ER * cos(theta * 180/(2*pi)) #These three can be rearranged to get different orientation
z3 = ER * sin(theta * 180/(2*pi))
a = ER/(1+ecc)
v3x = sqrt(userG*(m1+m2)*((2/ER)-(1/a)))
v3y = 0
v3z = 0;

In [15]:
xvals = [x1,x2,x3]
yvals = [y1,y2,y3]
zvals = [z1,z2,z3]
pxvals = [v1x*m1,v2x*m2,v3x*m3]
pyvals = [v1y*m1,v2y*m2,v3y*m3]
pzvals = [v1z*m1,v2z*m2,v3z*m3]

df = DataFrame(x = xvals,
                        y  = yvals,
                        z  = zvals,
                        px = pxvals,
                        py = pyvals,
                        pz = pzvals,
                        )
CSV.write(userpath * "/nbodyinitialData.csv", df)

LoadError: UndefVarError: DataFrame not defined

# Kozai Mechanism Initial Data

In [2]:
m1 = 1;
m2 = 1;
m3 = 1;        # Elliptical object
mbi = m1 + m2; # Mass of the binary
userG = 1;

# Inner Orbital Elements

aIn     = .1;            # Semimajor Axis
eIn     = .6;           # Eccentricity
iIn     = 30 *(pi/180); # Inclination (input degrees, comes out radians)
wIn     = pi *(pi/180); # Argument of pariastron (input degrees, comes out radians)
omegaIn = 0 *(pi/180);  # Longitude of ascending node (input degrees, comes out radians)
MIn     = 0 *(pi/180);  # Mean anomaly (input degrees, comes out radians)

# Outer Orbital Elements

aOut     = 10;           # Semimajor Axis
eOut     = 0;            # Eccentricity
iOut     = 0 *(pi/180);  # Inclination (input degrees, comes out radians)
wOut     = 0 *(pi/180);  # Argument of pariastron, undefined in circular orbits (input degrees, comes out radians)
omegaOut = 0 *(pi/180);  # Longitude of ascending node (input degrees, comes out radians)
MOut     = 20 *(pi/180); # Mean anomaly (input degrees, comes out radians)


# Determining the eccentric anomaly

tol0 = 10^-6; # Tolerance of eccentric anomaly evaluation

u0 = 10;

In [3]:
function initialdata(m1,m2,userG,a,e,i,w,omega,M,tol0,u0)
    
    mu=m1*m2/(m1+m2)
    
    tol  = 1;      # Initalizing iterative tolerance
    
    while abs(tol) > tol0
        u0 = u0 - (-M + u0 - e*sin(u0))/(1 - e*cos(u0))
        tol = -M + u0 - e*sin(u0)
    end

    # Using eccentric anomaly to find true anomaly

    f = atan(sin(u0)*sqrt(1-e^2)/(cos(u0)-e));

    # Solving for Polar Coordinates

    r     = a*(1-e^2)/(1-e*cos(f));
    phi   = omega + atan(tan(w+f)*cos(i));
    theta = acos(sin(w+f)*sin(i));

    # Some useful quantities in determining velocities

    gr     = a*(1-e^2)*e*sin(f)/(1+e*cos(f))^2;
    gtheta = (-1/sin(theta))*cos(w+f)*sin(i);
    gphi   = cos(phi-omega)^2*cos(i)/cos(w+f)^2;

    fdot   = sqrt(userG*mbi*((2/r)-(1/a))*1/(gr^2 + (r*gtheta)^2 + (r*sin(theta)*gphi)^2))

    # Setting up polar velocities

    rdot     = gr*fdot;
    thetadot = gtheta*fdot;
    phidot   = gphi*fdot;
    
    ### TODO: Convert Polar speeds and coordinates to cartesian
    #         Determine where exactly this function can be used
    #         Implement this function to get initial data
    #         Return correct data points
    #         Add conserved quantities
    ###
    
    x=r*sin(theta)*cos(phi)
    y=r*sin(theta)*sin(phi)
    z=r*cos(theta)
    
    # Double check math for everything past this point
    
    xdot = rdot*sin(theta)*cos(phi) + r*thetadot*cos(theta)*cos(phi) - phidot*r*sin(phi)*sin(theta)
    ydot = rdot*sin(theta)*sin(phi) + r*thetadot*cos(theta)*sin(phi) + r*phidot*sin(theta)*cos(phi)
    zdot = rdot*cos(theta) - r*thetadot*sin(theta)
    
    x1 = m2*x/(m1+m2)
    y1 = m2*y/(m1+m2)
    z1 = m2*z/(m1+m2)
    
    x2 = -m1*x/(m1+m2)
    y2 = -m1*y/(m1+m2)
    z2 = -m1*z/(m1+m2)
    
    px1 = mu*xdot
    py1 = mu*ydot
    pz1 = mu*zdot
    
    px2 = -mu*xdot
    py2 = -mu*ydot
    pz2 = -mu*zdot
    
    [[x1,y1,z1],[x2,y2,z2],[px1,py1,pz1],[px2,py2,pz2]]
    
    
    
    # Calculate angular momentum
    # linear momentum = mu * v
    # mv1 + mv2 = 0
    # Consider that c=1, G=1
    # Double check that using SI unites
    
end
    
    


initialdata (generic function with 1 method)

## Three Body - Finding values from KM Paper

In [20]:
valuesIn=initialdata(m1,m2,userG,aIn,eIn,iIn,wIn,omegaIn,MIn,tol0,u0);
valuesOut=initialdata(m3,mbi,userG,aOut,eOut,iOut,wOut,omegaOut,MOut,tol0,u0);

In [18]:
r1 = valuesIn[1]+valuesOut[2]
r2 = valuesIn[2]+valuesOut[2]
r3 = valuesOut[1]
p1 = valuesIn[3]+valuesOut[4]
p2 = valuesIn[4]+valuesOut[4]
p3 = valuesOut[3]

[r1,r2,r3,p1,p2,p3] # These are the final values for the initial data taken with the center of mass of the system as the origin

df = DataFrame(r10 = r1,
                        r20  = r2,
                        r30  = r3,
                        p10  = p1,
                        p20  = p2,
                        p30  = p3,
                        )
CSV.write(userpath * "/3BodyKozaiInitialData.csv", df)

6-element Array{Array{Float64,1},1}:
 [-3.0524289639639193, -1.1362702351197462, 0.0021921466059513827]
 [-3.2121885079421375, -1.143864053718042, -0.002192146605951791]
 [6.264617471906057, 2.2801342888377882, 4.082155997157844e-16]
 [0.040698345008973756, 0.6866284977602827, 0.5581768756088485]
 [0.16324306569780364, -1.2469529185688755, -0.5581768756088485]
 [-0.1019707053533887, 0.28016221040429645, 0.0]

# Two Body Circular Orbit Test

In [6]:
initialdata(m1,m2,userG,aIn,0.9,iIn,wIn,omegaIn,MIn,tol0,u0)

4-element Array{Array{Float64,1},1}:
 [0.09485722923706551, 0.0045088297927625655, 0.0026031740945816795]
 [-0.09485722923706551, -0.0045088297927625655, -0.0026031740945816795]
 [-0.028113687028940088, 0.44359399962562723, 0.2561091150947586]
 [0.028113687028940088, -0.44359399962562723, -0.2561091150947586]